In [4]:
import polars as pl

lf = pl.scan_parquet(
    "https://www.data.gouv.fr/fr/datasets/r/11cea8e8-df3e-4ed1-932b-781e2635e432"
)


def format_number(number) -> str:
    number = "{:,}".format(number).replace(",", " ")
    return number


# df = pl.read_parquet("/home/colin/git/decp-processing/dist/decp.parquet")

pl.Config(fmt_str_lengths=120, fmt_table_cell_list_len=50, set_tbl_rows=100)
var = "a"
print(pl.__version__)

1.36.1


In [2]:
lf_uid = (
    lf.select("uid", "montant", "acheteur_id", "donneesActuelles")
    .filter(pl.col("donneesActuelles"))
    .unique("uid")
)

df_uid = lf_uid.collect(engine="streaming")

lf_titu = lf.select(
    "uid", "donneesActuelles", "titulaire_id", "titulaire_typeIdentifiant"
).filter(pl.col("donneesActuelles"))
df_titu = lf_titu.collect(engine="streaming")

print("somme", format_number(int(df_uid.select(pl.sum("montant")).item())), "euros")

print("total marchés uniques : ", format_number(df_uid.height))
print("marchés/titulaires actuels (lignes) : ", format_number(df_titu.height))
print("total acheteurs uniques : ", format_number(df_uid.unique("acheteur_id").height))
print(
    "total titulaires uniques : ", format_number(df_titu.unique("titulaire_id").height)
)


# for year in range(2018, 2025 + 1):
#     year = str(year)
#     print("# " + year)
#     print(
#         "  - marches :".ljust(20, " "),
#         lf.filter(pl.col("dateNotification").dt.year() == int(year))
#         .unique("uid")
#         .height,
#     )
#     print(
#         "  - acheteurs :".ljust(20, " "),
#         lf.filter(pl.col("dateNotification").dt.year() == int(year))
#         .unique("acheteur_id")
#         .height,
#     )
#     print(
#         "  - tiul :".ljust(20, " "),
#         lf.filter(pl.col("dateNotification").dt.year() == int(year))
#         .unique("titulaire_id")
#         .height,
#     )

somme 10 207 777 477 970 euros
total marchés uniques :  1 539 892
marchés/titulaires actuels (lignes) :  1 782 501
total acheteurs uniques :  26 773
total titulaires uniques :  193 799


In [3]:
lf_datasets = (
    lf.select("uid", "sourceDataset")
    .unique()
    .group_by("sourceDataset")
    .len()
    .sort("len", descending=True)
)

print(lf_datasets.collect(engine="streaming"))

# df = df.filter(pl.col("datePublicationDonnees").cast(pl.String) > "2018").with_columns(
#     (pl.col("datePublicationDonnees").dt.year().cast(pl.String) + "_" +
#      pl.col("datePublicationDonnees").dt.month().cast(pl.String).str.pad_start(2, "0"))
#     .alias("moisPublicationDonnees"))
# print("aws par mois", df.filter(pl.col("sourceDataset") == "datagouv_aws").
#       select("uid", "moisPublicationDonnees").unique("uid")
#       .group_by("moisPublicationDonnees").len()
#      .sort(by="moisPublicationDonnees"))

shape: (52, 2)
┌─────────────────────────────────┬────────┐
│ sourceDataset                   ┆ len    │
│ ---                             ┆ ---    │
│ str                             ┆ u32    │
╞═════════════════════════════════╪════════╡
│ aife_dume                       ┆ 322748 │
│ aws_marches-publics.info_legacy ┆ 267160 │
│ pes_marche_2024                 ┆ 253292 │
│ pes_marche_legacy               ┆ 198750 │
│ e-marchespublics.com_dematis    ┆ 101876 │
│ scrap_marches-publics.info      ┆ 75679  │
│ atexo_maximilien                ┆ 56777  │
│ aws_marches-publics.info        ┆ 48855  │
│ megalis_bretagne                ┆ 41747  │
│ scrap_marches-securises.fr      ┆ 38845  │
│ scrap_aife_dume                 ┆ 28140  │
│ atexo_mp_aquitaine              ┆ 24759  │
│ arnia                           ┆ 24394  │
│ atexo_mp_596280                 ┆ 17643  │
│ atexo_marseille                 ┆ 9471   │
│ atexo_france_tv                 ┆ 7670   │
│ atexo_mp_ain                    ┆ 5910